In [5]:
import os
import sys
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Import our updated prompt functions and utilities
from prompt_utils import create_context_aware_prompt, create_non_context_aware_prompt
from llm_utils import TEXT_MODEL, is_student_rude, log_conversation

load_dotenv(dotenv_path=os.path.join('..', '.env'))

print("Setup complete.")


Setup complete.


In [2]:
class Chatbot:
    def __init__(self, name: str, system_prompt: str, log_dir: str, draft_id: int):
        self.name = name
        self.system_prompt = system_prompt
        self.history = [{'role': 'user', 'parts': [self.system_prompt]}]
        
        # Setup logging
        log_subdir = os.path.join('../logs', log_dir)
        self.log_path = os.path.join(log_subdir, f"conversation_draft_{draft_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
        
        # --- UI Component Changes ---
        # Use Textarea for the chat log to get automatic text wrapping
        self.chat_log = widgets.Textarea(
            value="",
            layout={'border': '1px solid black', 'height': '300px', 'width': '98%'},
            disabled=True # Make it read-only
        )
        self.input_box = widgets.Text(placeholder='Type here and press Enter...', layout={'width': '99%'})
        
        # Assemble the UI into a vertical box with a title
        self.ui = widgets.VBox([widgets.Label(value=self.name), self.chat_log, self.input_box], layout={'width': '50%'})
        
        # Link event handler
        self.input_box.on_submit(self.handle_send)

    def _append_to_log(self, new_text: str):
        """Helper function to append text to the Textarea widget."""
        self.chat_log.value += new_text

    def start_session(self):
        """Generates the AI's opening message and displays it."""
        self._append_to_log("Tutor is generating an opening message...\n")
            
        initial_response = TEXT_MODEL.generate_content(self.history)
        opening_message = initial_response.text
        
        self.history.append({'role': 'model', 'parts': [opening_message]})
        log_conversation(self.log_path, self.history)
        
        # Overwrite the "generating..." message with the final opening message
        self.chat_log.value = f"Tutor: {opening_message}\n"

    def handle_send(self, event):
        user_message = self.input_box.value
        if not user_message:
            return

        self._append_to_log(f"\nYou: {user_message}\n")

        if is_student_rude(user_message):
            rude_response = "It seems you're frustrated. Let's end our session here for today. Please remember to be respectful."
            self._append_to_log(f"\nTutor: {rude_response}\n")
            self.input_box.disabled = True
            return

        self.history.append({'role': 'user', 'parts': [user_message]})
        
        self._append_to_log("\nTutor is typing...\n")
        
        response = TEXT_MODEL.generate_content(self.history)
        ai_response = response.text
        
        # Overwrite the "typing..." message by replacing the last line
        lines = self.chat_log.value.strip().split('\n')
        lines[-1] = f"Tutor: {ai_response}"
        self.chat_log.value = "\n".join(lines) + "\n"

        self.history.append({'role': 'model', 'parts': [ai_response]})
        log_conversation(self.log_path, self.history)
        self.input_box.value = ""

print("Chatbot class updated with text wrapping and better UI handling.")

Chatbot class updated with text wrapping and better UI handling.


In [6]:

# --- Load the Student Context ---
analysis_path = '../results/student_drafts_with_analysis.csv'
try:
    df_analysis = pd.read_csv(analysis_path)
    student_case = df_analysis.iloc[0]
    
    # --- 1. Display Problem Context AT THE TOP ---
    print("--- Problem Context ---")
    display(widgets.HTML(f"<b>Problem:</b> {student_case['problem_text']}"))
    display(widgets.HTML(f"<b>Student's Incorrect Answer:</b> {student_case['student_answer']} (Correct: {student_case['correct_answer']})"))
    display(widgets.HTML("<b>Student's Handwritten Draft:</b>"))
    display(widgets.Image(
        value=requests.get(student_case['student_draft_image_url']).content,
        format='png',
        width=400
    ))
    print("\n--- Tutoring Session Started ---")

    # --- 2. Create Prompts ---
    prompt_aware = create_context_aware_prompt(
        error_type=student_case['ai_error_type'],
        detailed_explanation=student_case['ai_detailed_explanation']
    )
    prompt_unaware = create_non_context_aware_prompt(
        problem_text=student_case['problem_text']
    )
    
    # --- 3. Instantiate Chatbots ---
    bot_aware = Chatbot(
        name="✅ Context-Aware Tutor (Knows Your Mistake)",
        system_prompt=prompt_aware,
        log_dir="context_aware",
        draft_id=student_case['draft_id']
    )
    
    bot_unaware = Chatbot(
        name="❓ Non-Context-Aware Tutor (Doesn't Know Your Mistake)",
        system_prompt=prompt_unaware,
        log_dir="non_context_aware",
        draft_id=student_case['draft_id']
    )
    
    # --- 4. Start Sessions & Display UI ---
    bot_aware.start_session()
    bot_unaware.start_session()
    
    # Use HBox to display the two UIs next to each other
    chat_ui = widgets.HBox([bot_aware.ui, bot_unaware.ui])
    display(chat_ui)
    
    print(f"\nLogs for the left tutor will be in: {os.path.dirname(bot_aware.log_path)}")
    print(f"Logs for the right tutor will be in: {os.path.dirname(bot_unaware.log_path)}")
    
except FileNotFoundError:
    print(f"Error: Analysis file not found at {analysis_path}. Please run Notebook 01 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

--- Problem Context ---


HTML(value='<b>Problem:</b> A number is 35 less than 24 times 145. What is this number?')

HTML(value="<b>Student's Incorrect Answer:</b> 3345 (Correct: 3445)")

HTML(value="<b>Student's Handwritten Draft:</b>")

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0b@\x00\x00\x06f\x08\x02\x00\x00\x00\xb4\x0f\xb2\xd…


--- Tutoring Session Started ---


/var/folders/bt/_z83bhz94vzfbl2gms80vwmh0000gn/T/ipykernel_21763/1838196061.py:24: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.input_box.on_submit(self.handle_send)
E0000 00:00:1758319855.190364 31753641 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



Logs for the left tutor will be in: ../logs/context_aware
Logs for the right tutor will be in: ../logs/non_context_aware
